# VGG-16

In this notebook, we implement VGG-16 using Keras. 

Refer to paper [VERY DEEP CONVOLUTIONAL NETWORKS FOR LARGE-SCALE IMAGE RECOGNITION](https://arxiv.org/pdf/1409.1556.pdf) for the architecture of VGG-16.

## 1. Figure out VGG-16 architecture 

Figure below shows various VGG architectures. We will focus on column `D` which is the configuration for VGG-16.

<img src='images/vgg_architectures.png'>

In the table description, we can understand that `conv3–64` is a convolutional layer with a `receptive field` of size 3x3 and 64 channels (filters):

> The convolutional layer parameters are denoted as "conv⟨receptive field size⟩-⟨number of channels⟩"

To find convolutional padding and stride, we read through the paper:

<img src='images/vgg_parameters_description.png'>

Now we know everything about the network architecture:

* input size: 224 x 224;
* the receptive field size is 3 x 3;
* the convolution stride is 1 pixel;
* the padding is 1 (for receptive field of 3 x 3) so we keep the same spatial resolution;
* the max pooling is 2 x 2 with stride of 2 pixels;
* there are two fully connected layers with 4096 units each;
* the last layer is a softmax classification layer with 1000 units (representing the 1000 ImageNet classes);
* the activation function is the ReLU

We can now calculate the number of learnable parameters. You can find this information in Section 2.3 (Discussion) of this paper.

For the first convolutional layer, the network has to learn 64 filters with size 3x3 along the input depth 3. Plus, each one of the 64 filters has a bias, so the total number of parameters is $64\times3\times3\times3 + 64 = 1792$. You can apply the same logic for other convolutional layers.

The depth of a layer’s output will be the number of its convolutional filters. The padding is chosen as 1 pixel so the spatial resolution is preserved through the convolutional layers. Thus, the spatial resolution will only change at the pooling layers. So, the output of the first convolutional layer will be $224 \times 224 \times 64$.

> For how to calculate the output shape of a convolution layer, please refer to [this page](http://cs231n.github.io/convolutional-networks/). Basically, we are using the formula $(W - F +2P)/S + 1$ to calculate hte output shape.

The pooling layer does not learn anything, so we have 0 learnable parameters in it. To calculate the output shape of the pooling layer, we have to consider the size of the window and the stride. Since the window is 2 x 2 and the stride is 2, the layer is outputting a pixel for every 2 x 2 pixels and jumping by 2 pixels to do the next calculation (no overlap occurs), so the spatial resolution is divided by 2 in each pooling layer. The depth remains the same.

To calculate the number of parameters in the fully-connected layers, we have to multiply the number of units in the previous layer by the number of units in the current layer. By following the logic presented in the previous paragraphs, you can see that the number of units in the last convolutional layer will be 7x7x512. So, the total number of parameters in the first fully-connected layer will be $7\times7\times512\times4096 + 4096 = 102764544$.

## 2. Define VGG-16 Model using Keras 

We now know all the hyperparameters we need to know to configure the architecture of VGG-16. Then, we need to figure out how to define those layers using Keras.

We first define an [Input](https://keras.io/layers/core/#input) with shape ($224 \times 224 \times 3$) according to the paper.

By reading the [Conv2D document](https://keras.io/layers/convolutional/), we need to specify arguments: `filters`, `kernel_size`, `strides`, `padding`, and `activation`. 

Setting the filters, the kernel size and the stride is trivial. Setting the activation function requires that you go to the [Activations documentation](https://keras.io/activations/).

There are two options for the padding: `valid` or `same`. 
* With border mode “valid” you get an output that is smaller than the input because the convolution is only computed where the input and the filter fully overlap.
* With border mode “same” you get an output that is the “same” size as the input. That means that the filter has to go outside the bounds of the input by “filter size / 2” — the area outside of the input is normally padded with zeros.

So, we want the padding to be set as same.

For the [MaxPooling2D layer](https://keras.io/layers/pooling/), we need to set the `stride`, the `pooling size` and the `padding`. Since we want a padding of zero, we just use padding="valid". As valid is the default value for padding, we can omit this argument (but note that Keras API change a lot, so this can cause a change in the architecture for future versions of Keras).

Before we call the fully-connected ([Dense layers](https://keras.io/layers/core/#dense)), we need to [flatten](https://keras.io/layers/core/#flatten) the output of the last convolutional networks. This will just reshape the 3D output of the convnet to 1D.

Finally, for the Dense layers, we just have to set the number of units and the activation function.

In [7]:
from keras.models import Model
from keras.layers import Input, Dense, Activation, Dropout, Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D

input_ = Input(shape = (224, 224, 3))

x = Conv2D(64, (3, 3), padding='same', activation='relu')(input_)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)

x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)

x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)

x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)

x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)

x = Flatten()(x)
x = Dense(4096, activation='relu')(x)
x = Dense(4096, activation='relu')(x)
x = Dense(1000, activation='softmax')(x)

model = Model(inputs=input_, outputs=x)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 56, 56, 128)       0         
__________

### Compare our implementation with Keras built-in VGG-16

In [2]:
from keras.applications import VGG16, VGG19

VGG16(weights=None).summary()
# VGG19().summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

## Reference

[Reading the VGG Network Paper and Implementing It From Scratch with Keras](https://hackernoon.com/learning-keras-by-implementing-vgg16-from-scratch-d036733f2d5)